In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
from fradm.tn_test import *
import matplotlib.pyplot as plt
from ncon import ncon

In [21]:
L = 7
chi = 8
chain = Z2_chain_massive_mps(L=L, d=2, chi=chi, h1=0.1, h2=0.1)
chain._random_state(seed=3, chi=chi)
chain.canonical_form()
chain.DMRG(trunc_tol=False, trunc_chi=True)
# chain.DMRG(trunc_tol=True, trunc_chi=False)
tensor_shapes(chain.sites)
Z = np.array([[1,0],[0,-1]])
Z = np.array([[1,0],[0,-1]])
exp_vals = []
for i in range(L):
    chain.local_order_param(site=i)
    occup = chain.mpo_first_moment().real
    exp_vals.append(occup)

Sweep n: 0

Sweep n: 1

##############################
The energy between the two last updated states converged
to an order of 1e-10 after:
1 sweeps at site 2
total iterations 13
total time: 0.1002027639769949
##############################
(1, 2, 2)
(2, 2, 4)
(4, 2, 8)
(8, 2, 8)
(8, 2, 4)
(4, 2, 2)
(2, 2, 1)


In [26]:
occup_state = np.array([0,1]).reshape((1,2,1))
vacuum_state = np.array([1,0]).reshape((1,2,1))
vacuum_tensor = [vacuum_state for _ in range(L)]
vacuum_tensor[L//2] = occup_state
vacuum_tensor[L//2-1] = occup_state
vacuum_tensor[L//2+1] = occup_state
perturbed_tensor = vacuum_tensor

In [28]:
chain = Z2_chain_massive_mps(L=L, d=2, chi=chi, h1=0.1, h2=0.1)
chain.sites = perturbed_tensor
# chain.enlarge_chi()
# chain.TEBD_variational()

(1, 2, 8)
(8, 2, 8)
(8, 2, 8)
(8, 2, 8)
(8, 2, 8)
(8, 2, 8)
(8, 2, 1)


[(1, 2, 8), (8, 2, 8), (8, 2, 8), (8, 2, 8), (8, 2, 8), (8, 2, 8), (8, 2, 1)]

## Construct MPO

We define three mpos for the mass and gauge sites, we have to use them to find a single mpo for the bulk of the chain and the boundaries

In [27]:
theta = np.pi/3
I = identity(2).toarray()
O = np.zeros((2,2))
X = sparse_pauli_x(0,1).toarray()
w_start = np.asarray([(np.cos(theta))**(1/3)*I, (np.sin(theta))**(1/3)*X]).reshape((1,2,2,2))
w_middle = np.asarray([[(np.cos(theta))**(1/3)*I, O],[O, (np.sin(theta))**(1/3)*X]])
w_end = np.asarray([(np.cos(theta))**(1/3)*I, -1j*(np.sin(theta))**(1/3)*X]).reshape((2,1,2,2))
print(np.cos(theta)**(1/3)*np.sin(theta)**(1/3),np.cos(theta)**(2/3),np.sin(theta)**(2/3))

0.7565428747114508 0.6299605249474367 0.9085602964160698


In [33]:
w_2_l = ncon([w_start,w_middle],[[-1,-3,-5,1],[-2,-4,1,-6]]).reshape((2,4,2,2))
print(w_2_l)

[[[[0.62996052 0.        ]
   [0.         0.62996052]]

  [[0.         0.        ]
   [0.         0.        ]]

  [[0.         0.75654287]
   [0.75654287 0.        ]]

  [[0.         0.        ]
   [0.         0.        ]]]


 [[[0.         0.        ]
   [0.         0.        ]]

  [[0.         0.75654287]
   [0.75654287 0.        ]]

  [[0.         0.        ]
   [0.         0.        ]]

  [[0.9085603  0.        ]
   [0.         0.9085603 ]]]]


In [30]:
w_1_l = w_start
print(w_1_l)

[[[[0.79370053 0.        ]
   [0.         0.79370053]]

  [[0.         0.95318429]
   [0.95318429 0.        ]]]]


In [32]:
w_1_r = w_end
print(w_1_r)

[[[[0.79370053+0.j         0.        +0.j        ]
   [0.        +0.j         0.79370053+0.j        ]]]


 [[[0.        +0.j         0.        -0.95318429j]
   [0.        -0.95318429j 0.        +0.j        ]]]]


In [34]:
w_2_r = ncon([w_middle,w_end],[[-1,-3,-5,1],[-2,-4,1,-6]]).reshape((4,2,2,2))
print(w_2_r)

[[[[0.62996052+0.j         0.        +0.j        ]
   [0.        +0.j         0.62996052+0.j        ]]

  [[0.        +0.j         0.        +0.j        ]
   [0.        +0.j         0.        +0.j        ]]]


 [[[0.        +0.j         0.        -0.75654287j]
   [0.        -0.75654287j 0.        +0.j        ]]

  [[0.        +0.j         0.        +0.j        ]
   [0.        +0.j         0.        +0.j        ]]]


 [[[0.        +0.j         0.        +0.j        ]
   [0.        +0.j         0.        +0.j        ]]

  [[0.        +0.j         0.75654287+0.j        ]
   [0.75654287+0.j         0.        +0.j        ]]]


 [[[0.        +0.j         0.        +0.j        ]
   [0.        +0.j         0.        +0.j        ]]

  [[0.        -0.9085603j  0.        +0.j        ]
   [0.        +0.j         0.        -0.9085603j ]]]]


In [70]:
w_b_1 = ncon([w_start,w_middle,w_end],[[-1,-4,-7,1],[-2,-5,1,2],[-3,-6,2,-8]]).reshape((4,4,2,2))
w_b_1

array([[[[0.5       +0.j        , 0.        +0.j        ],
         [0.        +0.j        , 0.5       +0.j        ]],

        [[0.        +0.j        , 0.        +0.j        ],
         [0.        +0.j        , 0.        +0.j        ]],

        [[0.        +0.j        , 0.60046848+0.j        ],
         [0.60046848+0.j        , 0.        +0.j        ]],

        [[0.        +0.j        , 0.        +0.j        ],
         [0.        +0.j        , 0.        +0.j        ]]],


       [[[0.        +0.j        , 0.        -0.60046848j],
         [0.        -0.60046848j, 0.        +0.j        ]],

        [[0.        +0.j        , 0.        +0.j        ],
         [0.        +0.j        , 0.        +0.j        ]],

        [[0.        -0.72112479j, 0.        +0.j        ],
         [0.        +0.j        , 0.        -0.72112479j]],

        [[0.        +0.j        , 0.        +0.j        ],
         [0.        +0.j        , 0.        +0.j        ]]],


       [[[0.        +0.j        , 0.

We can visualize it with `sympy`

In [46]:
from sympy import *
theta, I, O, X, i = symbols('θ I O X i')

In [68]:
w_l1 = Matrix([[(cos(theta))**(S(1)/3)* I, (sin(theta))**(S(1)/3)*X]])
w_l2 = Matrix([[(cos(theta))**(S(2)/3)* I, O, (sin(theta)*cos(theta))**(S(1)/3)*X, O],[O, (sin(theta)*cos(theta))**(S(1)/3)*X, O, (sin(theta))**(S(2)/3)*I]])
w_r1 = Matrix([[(cos(theta))**(S(1)/3)* I],[-i*(sin(theta))**(S(1)/3)*X]])
w_r2 = Matrix([[(cos(theta))**(S(2)/3)* I, O],
                [-i*(sin(theta)*cos(theta))**(S(1)/3)*X, O],
                [O, (sin(theta)*cos(theta))**(S(1)/3)*X],
                [O, -(sin(theta))**(S(2)/3)*I]])

In [67]:
display(w_l1,w_l2,w_r1,w_r2)

Matrix([[I*cos(θ)**(1/3), X*sin(θ)**(1/3)]])

Matrix([
[I*cos(θ)**(2/3),                        O, X*(sin(θ)*cos(θ))**(1/3),               O],
[              O, X*(sin(θ)*cos(θ))**(1/3),                        O, I*sin(θ)**(2/3)]])

Matrix([
[   I*cos(θ)**(1/3)],
[-X*i*sin(θ)**(1/3)]])

Matrix([
[            I*cos(θ)**(2/3),                        O],
[-X*i*(sin(θ)*cos(θ))**(1/3),                        O],
[                          O, X*(sin(θ)*cos(θ))**(1/3)],
[                          O,         -I*sin(θ)**(2/3)]])